In [1]:
%%capture
!pip install numpy==1.21.4
!pip install pandas==1.3.4
!pip install loguru==0.5.3
!pip install psycopg2-binary
!pip install pandarallel

In [43]:
import sys
import os
import gc
from tqdm import tqdm
import pandas as pd
from adding_tables_psycopg import AddingDataPsycopg
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False, nb_workers=os.cpu_count(), use_memory_fs=False)
tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [74]:
result_dir = './result'
merge_dir = './merge'
os.makedirs(merge_dir, exist_ok=True)

In [75]:
regions = [i for i in os.listdir() if (len(i) == 2) and (not i in ['88', '81', '82', '80', '84', '85'])]

In [62]:
def cleanup(x):
    del x
    gc.collect()

def expand_chain(row):
        levchain = eval(row['levchain'])
        for i in levchain:
            if row[f'l{i}'] in adobj.index:
                obj = adobj.loc[row[f'l{i}']]
                if len(obj):
                    if isinstance(obj, pd.DataFrame):
                        obj = obj.iloc[0]
                    row[f'l{i}_name'] = obj['NAME']
                    if not f'{i}' in ['1', '10', '9', '8', '7']:
                        row[f'l{i}_typename'] = obj['TYPENAME']
                        row[f'l{i}_OKTMO'] = obj['OKTMO']

        if pd.notna(row["id_tow"]):
            if row['id_tow'] in adobj.index:
                obj = adobj.loc[row['id_tow']]
                if len(obj):
                    if isinstance(obj, pd.DataFrame):
                        obj = obj.iloc[0]
                    row['tow_name'] = obj['NAME']
                    row['tow_typename'] = obj['TYPENAME']
                    row['tow_OKTMO'] = obj['OKTMO']              

        if pd.notna(row["id_mun"]):
            if row['id_mun'] in adobj.index:
                obj = adobj.loc[row['id_mun']]
                if len(obj):
                    if isinstance(obj, pd.DataFrame):
                        obj = obj.iloc[0]
                    row['mun_name'] = obj['NAME']
                    row['mun_typename'] = obj['TYPENAME']
                    row['mun_OKTMO'] = obj['OKTMO']   

        if pd.notna(row["street"]):
            if row[f'l{row["street"]}'] in adobj.index:
                obj = adobj.loc[row[f'l{row["street"]}']]
                if len(obj):
                    if isinstance(obj, pd.DataFrame):
                        obj = obj.iloc[0]
                    row['street_name'] = obj['NAME']

        row.drop(['chain', 'leftover'], inplace=True)
        return row

In [73]:
for r in tqdm(regions):
      if not (f'{r}_flat.csv' in os.listdir(merge_dir)):
            if (f'{r}_hadobj.csv' in os.listdir(result_dir)) and (f'{r}_parsed_chains.csv' in os.listdir(result_dir)):
                adobj = pd.read_csv(os.path.join(result_dir, f'{r}_hadobj.csv'), dtype={#'isactive': 'Int64', 'isactual': 'Int64',
                                                                'OBJECTID' : 'Int64',
                                                                'level': 'Int64', 'objectid': 'Int64',
                                                                'OKTMO' : 'str', 'KLADR' : 'str'}) \
                .set_index(['OBJECTID'])
                parsed_chains = pd.read_csv(os.path.join(result_dir, f'{r}_parsed_chains.csv'), dtype={"l1": 'Int64', "l2": 'Int64', "l3": 'Int64',
                                                                             "l4": 'Int64', "l5": 'Int64', "l6": 'Int64',
                                                                             "l7": 'Int64', "l8": 'Int64', "l9": 'Int64',
                                                                             "l10": 'Int64',
                                                                             'tow': 'Int64',
                                                                             'mun': 'Int64',
                                                                             'id_tow': 'Int64',
                                                                             'id_mun': 'Int64',
                                                                             'id_reg': 'Int64',
                                                                             'street': 'Int64'})   
                merged = parsed_chains.parallel_apply(expand_chain, axis=1)
                #merged = parsed_chains.progress_apply(expand_chain, axis=1)
                merged.to_csv(os.path.join(merge_dir, f'{r}_flat.csv'), index=False)
                cleanup(adobj)
                cleanup(parsed_chains)
                cleanup(merged)
                break

  0%|          | 0/1 [01:01<?, ?it/s]


In [76]:
sample = AddingDataPsycopg()
sample.create_table('merged_chains', 'gar_source')

total_parsed_chains = 0

for r in tqdm(regions):
    if f'{r}_flat.csv' in os.listdir(merge_dir):     
        merged_chains = pd.read_csv(os.path.join(merge_dir, f'{r}_flat.csv'), dtype={
                                                                     "l1": 'Int64', "l2": 'Int64', "l3": 'Int64',
                                                                     "l4": 'Int64', "l5": 'Int64', "l6": 'Int64',
                                                                     "l7": 'Int64', "l8": 'Int64', "l9": 'Int64',
                                                                     "l10": 'Int64',
                                                                     'tow': 'Int64',
                                                                     'mun': 'Int64',
                                                                     'id_tow': 'Int64',
                                                                     'id_mun': 'Int64',
                                                                     'id_reg': 'Int64',
                                                                     'street': 'Int64',
                                                                     "l1_OKTMO": 'str', "l2_OKTMO": 'str', "l3_OKTMO": 'str',
                                                                     "l4_OKTMO": 'str', "l5_OKTMO": 'str', "l6_OKTMO": 'str',
                                                                     "l7_OKTMO": 'str', "l8_OKTMO": 'str', "l9_OKTMO": 'str',
                                                                     "l10_OKTMO": 'str',
                                                                     'tow_OKTMO': 'str',
                                                                     'mun_OKTMO': 'str'
                                                                    })  
        total_parsed_chains += len(merged_chains)
        sample.write_to_sql(merged_chains, 'merged_chains', 'gar_source')
        cleanup(merged_chains)
    #break
print(f'total_parsed_chains: {total_parsed_chains}')

Вы подключены к базе: temp_inid


100%|██████████| 86/86 [18:03<00:00, 12.60s/it]

total_parsed_chains: 32448994
